In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import warnings 
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier as model
import pickle
import math
import itertools
import joblib
from category_encoders.james_stein import JamesSteinEncoder
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier
from sklearn.metrics import jaccard_score
from sklearn.ensemble import ExtraTreesClassifier
from catboost import CatBoostClassifier
import seaborn as sns
import gdal
from numpy import hstack
import sklearn.metrics as metrics
import tensorflow as tf 
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model,Sequential
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam, RMSprop, SGD 
from tensorflow.keras.layers import Dense, Input, GlobalMaxPool1D, Conv1D, MaxPooling1D, Embedding,LSTM, Bidirectional, Dropout, Flatten,BatchNormalization
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import sklearn.metrics as metrics
from joblib import Parallel, delayed
from functools import reduce
from imblearn.over_sampling import SVMSMOTE
from imblearn.under_sampling import OneSidedSelection
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
import rioxarray as rio
import xarray as xr
plt.rcParams["font.family"] = "Times New Roman"
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv(r'F:/tif data/final_data.csv')

In [ ]:
print('Landslides are', round(df['label'].value_counts()[1]/len(df) * 100,2), '% of the dataset')
print('No Landslides are', round(df['label'].value_counts()[0]/len(df) * 100,2), '% of the dataset')

In [ ]:
ndf.sample(frac=1, random_state=42).drop(['x','y'],axis=1,inplace=True)
new_df=ndf.copy()
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200,n_jobs=-1)
X, y = undersample.fit_resample(new_df.drop('label',axis=1), new_df['label'])
X['label']=y
print('Landslides are', round(X['label'].value_counts()[1]/len(X) * 100,2), '% of the dataset')
print('No Landslides are', round(X['label'].value_counts()[0]/len(X) * 100,2), '% of the dataset')
X.to_csv('./train_OSS.csv',index=False)

In [ ]:
pipeline = Pipeline(steps=[('o', over)])
X_train, y_train = pipeline.fit_resample(train.drop('label',axis=1), train['label'])
print('Resample dataset shape', Counter(y_train))
X_train['label']=y_train
X_train.to_csv(r'F:/tif data/training_data.csv',index=False)

In [ ]:
train=pd.read_csv(r'F:/python_nb/training_data.csv')
test=pd.read_csv(r'F:/tif data/test.csv')

In [ ]:
f = plt.figure(figsize=(44,20),dpi=300)# Entire DataFrame
ax = plt.axes()
X_train=train.drop(['x','y','label'],axis=1)
d={'road':'Road','aspec':'Aspect','sand':'Sand','silt':'Silt','river':'River'}
X_train.rename(d,axis=1,inplace=True)
corr = X_train.corr()

sns.heatmap(corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax)
ax.set_title("Correlation Matrix", fontsize=14)
plt.savefig('corr.jpg',dpi=300)
plt.show()

In [ ]:
test=pd.read_csv(r'F:/tif data/test.csv')
X_train=train.drop(['x','y','label'],axis=1)
y_train=train['label']

X_test=test.drop(['x','y','label'],axis=1)
y_test=test['label']
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values
min_max_scaler = MinMaxScaler().fit(X_train) # Fit On Training Data
X_train = min_max_scaler.transform(X_train) # Transform On Training Data
X_test= min_max_scaler.transform(X_test) # Transform On Validation Data
model = RandomForestClassifier(n_jobs=-1)

model.fit(X_train,y_train)

In [ ]:
d={'road':'Road','aspec':'Aspect','sand':'Sand','silt':'Silt','river':'River'}
test.rename(d,axis=1,inplace=True)

In [ ]:
fig=plt.figure(figsize=(10,8))
ax=plt.axes()
feature_scores = pd.Series(model.feature_importances_, index=test.drop(['x','y','label'],axis=1).columns).sort_values(ascending=False)
sns.barplot(x=feature_scores, y=feature_scores.index,color='grey')
plt.xlabel('Feature Importance Score',fontsize=20)
plt.ylabel('Landslide Conditioning Factors',fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.tick_params(axis='both', which='minor', labelsize=18)
plt.savefig(r'F:\paper1img/feature_importance.jpg',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
def build_model():
    
    # Sequence level Input Model
    
    Sequence_Input_tensor = Input(shape=(X_train.shape[1],),name='Sequence_input_tensor')


    x1=layers.Dense(units=16,
                      activation='relu',
                      kernel_regularizer=keras.regularizers.l1_l2(l1=0,l2=0.01), 
                      bias_regularizer=keras.regularizers.l1_l2(l1=0,l2=0.01),
                      kernel_initializer= tf.keras.initializers.he_normal(seed=700),
                      bias_initializer=tf.keras.initializers.Zeros()   )(Sequence_Input_tensor) 
    

    
    x1=layers.Dense(units=8,
                      activation='relu',
                      kernel_regularizer=keras.regularizers.l1_l2(l1=0,l2=0.01), 
                      bias_regularizer=keras.regularizers.l1_l2(l1=0,l2=0.01),
                      kernel_initializer= tf.keras.initializers.he_normal(seed=700),
                      bias_initializer=tf.keras.initializers.Zeros()   )(x1)
    
    
    
    x1=layers.Dense(units=4,
                      activation='relu',
                      kernel_regularizer=keras.regularizers.l1_l2(l1=0,l2=0.01), 
                      bias_regularizer=keras.regularizers.l1_l2(l1=0,l2=0.01),
                      kernel_initializer= tf.keras.initializers.he_normal(seed=700),
                      bias_initializer=tf.keras.initializers.Zeros()   )(x1)
    
    x1=layers.BatchNormalization()(x1)



    
    # Output
    output_tensor = layers.Dense(1,
                          activation='sigmoid',
                          kernel_initializer= tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=700),
                          bias_initializer=tf.keras.initializers.Zeros()) (x1)
    

    Basic_Model = Model(inputs=[Sequence_Input_tensor], outputs=output_tensor)

    
    Basic_Model.compile(optimizer=Adam(10**-5), loss='binary_crossentropy', metrics=['accuracy'])
    return Basic_Model

In [ ]:
batch_size = 16
epochs = 30
Training_scores= list()
callbacks = [tf.keras.callbacks.EarlyStopping('val_loss',patience=40,restore_best_weights=True)]  # Callbacks

with tf.device('/GPU:0'):
    model_1=build_model()
    history_1=model_1.fit(X_train,y_train, batch_size=batch_size, epochs=epochs,shuffle=True, verbose=2,validation_data=(X_test,y_test),callbacks=callbacks)

In [ ]:
def predict_classes(probability_array):
    for i in range(len(probability_array)):
        if (probability_array[i]>0.5):
            probability_array[i]=1
        else:
            probability_array[i]=0
    return (probability_array) 

In [ ]:
import math
def accuracy_check(pred,AY_VALIDATE):
    abc=confusion_matrix(AY_VALIDATE,pred,normalize='true')

    TN=abc[0][0]
    FP=abc[0][1]
    FN=abc[1][0]
    TP=abc[1][1]

    PS=FN+TP
    NS=TN+FP
    Accuracy           = ((TP+TN)/(PS+NS))*100
    Sensitivity        = (TP/PS)*100
    Precision          = (TP/(FP+TP))*100
    F1                 = (2*Sensitivity*Precision)/(Sensitivity+Precision )
    Speciﬁcity         = (TN/NS)*100
    BAC                = (Sensitivity + Speciﬁcity)/2
    MCC                = metrics.matthews_corrcoef(y_test,(model.predict(X_test)))
    print('Accuracy:' ,Accuracy)
    print('Sensitivity:', Sensitivity)
    print('Precision:' ,Precision)
    print('matrix',abc)  
    print(confusion_matrix(AY_VALIDATE,pred))
    MCC=(TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    print('MCC:',MCC)
    cmap=plt.cm.Greens
    title='Confusion matrix'
    cm=abc
    classes = ['No-Landslide', 'Landslide']
    normalize=True

    plt.figure(figsize = (10, 10))
    plt.imshow(cm, cmap=cmap)
    plt.title(title, size = 24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 14)
    plt.yticks(tick_marks, classes, size = 14)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    # Label the plot
    for i, j in itertools.product(range(cm.shape[0]),   range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), 
             fontsize = 20,
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
        
    # plt.grid()
    plt.tight_layout()
    plt.ylabel('True label', size = 18)
    plt.xlabel('Predicted label', size = 18)

In [ ]:
def get_models():
	models = list()
	models.append(('RF', RF(n_estimators=n_estimators,max_features='sqrt',n_jobs=-1)))
	models.append(('ANN', ANN())
	models.append(('svm', SVC(kernel=kernel,C=C,probability=True)))
	return models

def fit_ensemble(models, X_train, X_val, y_train, y_val):
	# fit all models on the training set and predict on hold out set
	meta_X = list()
	for name, model in models:
		# fit in training set
		model.fit(X_train, y_train)
		# predict on hold out set
		yhat = model.predict_proba(X_val)[:,1]
		# reshape predictions into a matrix with one column
		yhat = yhat.reshape(len(yhat), 1)
		# store predictions as input for blending
		meta_X.append(yhat)
	# create 2d array from predictions, each set is an input feature
	meta_X = hstack(meta_X)
	# define blending model
	blender = LogisticRegression()
	# fit on predictions from base models
	blender.fit(meta_X, y_val)
	return blender

def predict_ensemble(models, blender, X_test):
	# make predictions with base models
	meta_X = list()
	for name, model in models:
		# predict with base model
		yhat = model.predict_proba(X_test)[:,1]
		# reshape predictions into a matrix with one column
		yhat = yhat.reshape(len(yhat), 1)
		# store prediction
		meta_X.append(yhat)
	# create 2d array from predictions, each set is an input feature
	meta_X = hstack(meta_X)
	# predict
	return blender.predict_proba(meta_X)

X_train, X_val, y_train, y_val = train_test_split(X_TRAIN, Y_TRAIN, test_size=0.33, random_state=1)
X_test=X_TEST
Y_test=Y_TEST

print('Train: %s, Val: %s, Test: %s' % (X_train.shape, X_val.shape, X_test.shape))
# create the base models
models = get_models()
# train the blending ensemble
blender = fit_ensemble(models, X_train, X_val, y_train, y_val)
# make predictions on test set
yhat = predict_ensemble(models, blender, X_test)
# evaluate predictions
score = accuracy_score(Y_test, yhat)
print('Blending Accuracy: %.3f' % (score*100))